<a href="https://colab.research.google.com/github/xBosse/notebooks/blob/main/Modell_Datenerweiterung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Erst wird das Modell auf den schwach klassifizierten Daten trainiert und als Modell-Checkpoint gespeichet. Dieser Checkpoint wird mit den Shared Task daten nochmal trainiert.

# Imports

In [1]:
! pip install datasets transformers seqeval
import transformers, datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 25.5 MB/s 
     |████████████████████████████████| 4.2 MB 47.3 MB/s 
     |████████████████████████████████| 43 kB 741 kB/s 
     |████████████████████████████████| 1.1 MB 46.4 MB/s 
     |████████████████████████████████| 140 kB 59.8 MB/s 
     |████████████████████████████████| 212 kB 80.8 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 127 kB 55.9 MB/s 
     |████████████████████████████████| 6.6 MB 48.5 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 52.8 MB/s 
     |████████████████████████████████| 144 kB 74.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=e04935ecf7

# Datensätze laden

In [2]:
from datasets import load_dataset, load_metric, ClassLabel, Value, Features

import datasets

#DATENSATZ
train_url = "https://raw.githubusercontent.com/xBosse/datenerweiterung/main/train_texte.txt"

val_url = "https://raw.githubusercontent.com/xBosse/datenerweiterung/main/val_texte.txt"

test_url = "https://raw.githubusercontent.com/xBosse/datenerweiterung/main/test_texte.txt"


# eigener Code

classFeatures = datasets.Features({
    'words': datasets.Sequence(Value("string")),
    'subj': datasets.Sequence(ClassLabel(names=['O', 'B-SUBJ', 'I-SUBJ']))
})

# eigener Parameter

datasets = load_dataset('json', data_files = { 'train': train_url, 'validation': val_url, 'test': test_url}, features=classFeatures)

task = "subj_preTrained_with_noisyData" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-german-cased"

# bert-base-german-cased 

batch_size = 16

Using custom data configuration default-9a1f54ea1fcbc3ee


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9a1f54ea1fcbc3ee/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Tokenizer erstellen / Labels auf tokenisierte Wörteranzahl anpassen

In [3]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474k [00:00<?, ?B/s]

In [4]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["words"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"subj"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [5]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

# Trainer erstellen, Modell trainieren + Modell-checkppoint auf Huggingface hochladen

In [6]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

#LABELS # eigener Code
#label_list = ['O', 'B-SUBJ', 'I-SUBJ']
label_list = datasets["train"].features[f"subj"].feature.names

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-b

In [7]:


model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,                           ######### hyperparameter - epoch ##########
    weight_decay=0.01,
    push_to_hub=True,
)

In [8]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

metric = load_metric("seqeval")

In [9]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [10]:
! git config --global credential.helper store

from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [11]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [12]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2 into local empty directory.


In [13]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.038000,0.009059,0.969450,0.942638,0.955856,0.997358
2,0.007900,0.007420,0.977557,0.959283,0.968334,0.998096


Saving model checkpoint to bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/checkpoint-500
Configuration saved in bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/checkpoint-500/config.json
Model weights saved in bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/checkpoint-500/special_tokens_map.json
tokenizer config file saved in bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/tokenizer_config.json
Special tokens file saved in bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored

TrainOutput(global_step=1250, training_loss=0.019314336585998536, metrics={'train_runtime': 32155.8909, 'train_samples_per_second': 0.622, 'train_steps_per_second': 0.039, 'total_flos': 1770522003061344.0, 'train_loss': 0.019314336585998536, 'epoch': 2.0})

In [14]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1576
  Batch size = 16


{'epoch': 2.0,
 'eval_accuracy': 0.9980964516304475,
 'eval_f1': 0.968333979578648,
 'eval_loss': 0.00742042250931263,
 'eval_precision': 0.9775574112734864,
 'eval_recall': 0.9592829705505762,
 'eval_runtime': 762.0486,
 'eval_samples_per_second': 2.068,
 'eval_steps_per_second': 0.13}

In [15]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1576
  Batch size = 16


{'SUBJ': {'f1': 0.968333979578648,
  'number': 3905,
  'precision': 0.9775574112734864,
  'recall': 0.9592829705505762},
 'overall_accuracy': 0.9980964516304475,
 'overall_f1': 0.968333979578648,
 'overall_precision': 0.9775574112734864,
 'overall_recall': 0.9592829705505762}

In [16]:
trainer.push_to_hub()

Saving model checkpoint to bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2
Configuration saved in bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/config.json
Model weights saved in bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/pytorch_model.bin
tokenizer config file saved in bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/tokenizer_config.json
Special tokens file saved in bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/414M [00:00<?, ?B/s]

Upload file runs/May29_13-41-22_75316bbee9ff/events.out.tfevents.1653864619.75316bbee9ff.60.2: 100%|##########…

Upload file runs/May29_13-41-22_75316bbee9ff/events.out.tfevents.1653831700.75316bbee9ff.60.0:  63%|######3   …

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2
   5ebca74..ff65904  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.9775574112734864}, {'name': 'Recall', 'type': 'recall', 'value': 0.9592829705505762}, {'name': 'F1', 'type': 'f1', 'value': 0.968333979578648}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9980964516304475}]}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2
   ff65904..18189b5  main -> main



'https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/commit/ff6590425c666f4c46c856119cd9ab075a39b88c'

# Training mit Shared-Task Daten

# Datensätze laden

In [17]:
from datasets import load_dataset, load_metric, ClassLabel, Value, Features

import datasets

#DATENSATZ
train_url = "https://raw.githubusercontent.com/xBosse/datasets/main/train_shata14.txt"

val_url = "https://raw.githubusercontent.com/xBosse/datasets/main/val_shata14.txt"

test_url = "https://raw.githubusercontent.com/xBosse/datasets/main/test_steps16.txt"

# eigener Code

classFeatures = datasets.Features({
    'words': datasets.Sequence(Value("string")),
    'subj': datasets.Sequence(ClassLabel(names=['O', 'B-SUBJ', 'I-SUBJ']))
})

# eigener Parameter

datasets = load_dataset('json', data_files = { 'train': train_url, 'validation': val_url, 'test': test_url}, features=classFeatures)

task = "subj_noisy_Data_fine_tuned" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2"
#model_checkpoint = "bert-base-german-cased-finetuned-subj_preTrained_with_noisyData"
model_checkpoint2 = "bert-base-german-cased"

# bert-base-german-cased 

batch_size = 16

Using custom data configuration default-68ea738c7c1a98fd


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-68ea738c7c1a98fd/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Tokenizer erstellen / Labels auf tokenisierte Wörteranzahl anpassen

In [18]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmphtdt1il_


Downloading:   0%|          | 0.00/327 [00:00<?, ?B/s]

storing https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/5d47b116883399cf2ea6f29c287223a4cf5958d22ad3993b913389d25a741fff.b3c61ee6afa415acf7b5324431d5621913bec029083ab8c61d250a77afa70181
creating metadata file for /root/.cache/huggingface/transformers/5d47b116883399cf2ea6f29c287223a4cf5958d22ad3993b913389d25a741fff.b3c61ee6afa415acf7b5324431d5621913bec029083ab8c61d250a77afa70181
https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpu9teo21d


Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

storing https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/15e2ed81bfaf203f1e39993ffa9f11169d643b47c7558c9bb0a3356ef1286f5f.bbe879d3c449e63417039a6f8770844295dee2ca4128198cc726a470868e084f
creating metadata file for /root/.cache/huggingface/transformers/15e2ed81bfaf203f1e39993ffa9f11169d643b47c7558c9bb0a3356ef1286f5f.bbe879d3c449e63417039a6f8770844295dee2ca4128198cc726a470868e084f
https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp87ly36t0


Downloading:   0%|          | 0.00/709k [00:00<?, ?B/s]

storing https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/fd0e13b3a094a67135b4ffe80ecf254378113fed8078b5d4f72ec5084fa9b149.288fb93085b95ce3b4bb410604421b7ffec598459371b7a67447d4646308e86e
creating metadata file for /root/.cache/huggingface/transformers/fd0e13b3a094a67135b4ffe80ecf254378113fed8078b5d4f72ec5084fa9b149.288fb93085b95ce3b4bb410604421b7ffec598459371b7a67447d4646308e86e
https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmptcf8ocrv


Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

storing https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/12e3ae965f76308dd9eec48c02f5c1441c57b2c4792e6646a3ddb470d42cad97.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /root/.cache/huggingface/transformers/12e3ae965f76308dd9eec48c02f5c1441c57b2c4792e6646a3ddb470d42cad97.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/15e2ed81bfaf203f1e39993ffa9f11169d643b47c7558c9bb0a3356ef1286f5f.bbe879d3c449e63417039a6f8770844295dee2ca4128198cc726a470868e084f
loading file https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/tokenizer.json from cache at /root/.cache/hug

In [19]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["words"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"subj"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [20]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# Trainer erstellen, Modell trainieren + Modell-checkppoint auf Huggingface hochladen

In [21]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

#LABELS # eigener Code
#label_list = ['O', 'B-SUBJ', 'I-SUBJ']
label_list = datasets["train"].features[f"subj"].feature.names

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))  

https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpp9v56yao


Downloading:   0%|          | 0.00/799 [00:00<?, ?B/s]

storing https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/693192ae96edacccea6e8c1e1e147acf3b1a50cf41ed033a134cd005eddc539b.2296bbbb75ddf33020fde7bdd5aa665fb606923948cea3be86ba45e30729158b
creating metadata file for /root/.cache/huggingface/transformers/693192ae96edacccea6e8c1e1e147acf3b1a50cf41ed033a134cd005eddc539b.2296bbbb75ddf33020fde7bdd5aa665fb606923948cea3be86ba45e30729158b
loading configuration file https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/693192ae96edacccea6e8c1e1e147acf3b1a50cf41ed033a134cd005eddc539b.2296bbbb75ddf33020fde7bdd5aa665fb606923948cea3be86ba45e30729158b
Model config BertConfig {
  "_name_or_path": "tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2",
  "architectures": [
    "BertForTokenClas

Downloading:   0%|          | 0.00/414M [00:00<?, ?B/s]

storing https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/dbcc0f24452c5947009d67a926ae4009610fb0573d3a6821333fe80fe72c8568.15c2f33927854e59c2cc840ea21d5331913548fb92d1b69b2a1d1f6070562674
creating metadata file for /root/.cache/huggingface/transformers/dbcc0f24452c5947009d67a926ae4009610fb0573d3a6821333fe80fe72c8568.15c2f33927854e59c2cc840ea21d5331913548fb92d1b69b2a1d1f6070562674
loading weights file https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData_v2/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/dbcc0f24452c5947009d67a926ae4009610fb0573d3a6821333fe80fe72c8568.15c2f33927854e59c2cc840ea21d5331913548fb92d1b69b2a1d1f6070562674
All model checkpoint weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the 

In [22]:


model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    "bert-base-german-cased-noisy-pretrain-fine-tuned", #f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=7,                           ######### hyperparameter - epoch ##########
    weight_decay=0.01,
    push_to_hub=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

metric = load_metric("seqeval")

In [24]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [29]:
! git config --global credential.helper store

from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [30]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [31]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/bert-base-german-cased-noisy-pretrain-fine-tuned_v2 is already a clone of https://huggingface.co/tbosse/bert-base-german-cased-noisy-pretrain-fine-tuned_v2. Make sure you pull the latest changes with `repo.git_pull()`.


In [32]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 520
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 231


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.310542,0.773077,0.574286,0.659016,0.881336
2,No log,0.263151,0.758824,0.737143,0.747826,0.905530
3,No log,0.251678,0.763006,0.754286,0.758621,0.909562
4,No log,0.258994,0.814465,0.740000,0.775449,0.917051
5,No log,0.266537,0.793939,0.748571,0.770588,0.916475
6,No log,0.285432,0.795107,0.742857,0.768095,0.914747
7,No log,0.287152,0.786982,0.760000,0.773256,0.915899


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 58
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 58
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 58
  Batch size 

TrainOutput(global_step=231, training_loss=0.17859692594189663, metrics={'train_runtime': 2196.11, 'train_samples_per_second': 1.657, 'train_steps_per_second': 0.105, 'total_flos': 120671199441216.0, 'train_loss': 0.17859692594189663, 'epoch': 7.0})

In [33]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 58
  Batch size = 16


{'epoch': 7.0,
 'eval_accuracy': 0.9158986175115207,
 'eval_f1': 0.7732558139534884,
 'eval_loss': 0.28715187311172485,
 'eval_precision': 0.7869822485207101,
 'eval_recall': 0.76,
 'eval_runtime': 10.7179,
 'eval_samples_per_second': 5.412,
 'eval_steps_per_second': 0.373}

In [34]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 58
  Batch size = 16


{'SUBJ': {'f1': 0.7732558139534884,
  'number': 350,
  'precision': 0.7869822485207101,
  'recall': 0.76},
 'overall_accuracy': 0.9158986175115207,
 'overall_f1': 0.7732558139534884,
 'overall_precision': 0.7869822485207101,
 'overall_recall': 0.76}

In [35]:
trainer.push_to_hub()

Saving model checkpoint to bert-base-german-cased-noisy-pretrain-fine-tuned_v2
Configuration saved in bert-base-german-cased-noisy-pretrain-fine-tuned_v2/config.json
Model weights saved in bert-base-german-cased-noisy-pretrain-fine-tuned_v2/pytorch_model.bin
tokenizer config file saved in bert-base-german-cased-noisy-pretrain-fine-tuned_v2/tokenizer_config.json
Special tokens file saved in bert-base-german-cased-noisy-pretrain-fine-tuned_v2/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 3.34k/414M [00:00<?, ?B/s]

Upload file runs/May29_23-10-02_75316bbee9ff/1653865812.0167394/events.out.tfevents.1653865812.75316bbee9ff.60…

Upload file runs/May29_23-10-02_75316bbee9ff/events.out.tfevents.1653868046.75316bbee9ff.60.7: 100%|##########…

Upload file runs/May29_23-10-02_75316bbee9ff/1653865839.5078354/events.out.tfevents.1653865839.75316bbee9ff.60…

Upload file training_args.bin: 100%|##########| 3.17k/3.17k [00:00<?, ?B/s]

Upload file runs/May29_23-10-02_75316bbee9ff/events.out.tfevents.1653865839.75316bbee9ff.60.5:  46%|####5     …

Upload file runs/May29_23-10-02_75316bbee9ff/events.out.tfevents.1653865811.75316bbee9ff.60.3:  90%|########9 …

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/tbosse/bert-base-german-cased-noisy-pretrain-fine-tuned_v2
   5f5fb7d..5509881  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.7869822485207101}, {'name': 'Recall', 'type': 'recall', 'value': 0.76}, {'name': 'F1', 'type': 'f1', 'value': 0.7732558139534884}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9158986175115207}]}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/tbosse/bert-base-german-cased-noisy-pretrain-fine-tuned_v2
   5509881..1f93814  main -> main



'https://huggingface.co/tbosse/bert-base-german-cased-noisy-pretrain-fine-tuned_v2/commit/550988152868be7502c826374a7b75f40471fb1d'